Setup & Imports

In [ ]:
!pip install praw requests
import time
import praw
import requests

 Reddit API Initialization

In [45]:
# ✅ OpenRouter API key
OPENROUTER_API_KEY = (
    "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"
)

# ✅ Reddit API credentials (via PRAW)
reddit = praw.Reddit(
    client_id="5uBZvIgTKH5uE_jEJPIUeQ",
    client_secret="Tr8C5flzhU81yzryqWT7aL8XOtBxpQ",
    user_agent="u/Zealousideal_Wafer84"
)


Fetch Reddit Posts & Comments

In [46]:
def get_reddit_data(username, limit=30):
    """
    Fetch a Reddit user's most recent posts and comments.

    Returns:
        List[str]: Combined list of formatted posts and comments.
    """
    try:
        user = reddit.redditor(username)

        posts = [
            f"[POST] {submission.title}"
            for submission in user.submissions.new(limit=limit)
        ]

        comments = [
            f"[COMMENT] {comment.body}"
            for comment in user.comments.new(limit=limit)
        ]

        return posts + comments

    except Exception as err:
        print("❌ Error fetching Reddit data:", err)
        return []


Prompt Construction

In [47]:
def build_prompt(username, reddit_data):
    """
    Construct a structured UX-style LLM prompt from Reddit activity.

    Includes detailed formatting instructions for the model to return
    a user persona with behavioral sliders, motivations, goals, etc.
    """
    joined_data = "\n".join(reddit_data[:40])[:5000]

    prompt = f"""
You are a UX researcher creating user personas from Reddit activity.

Format the output to follow this structure and style.
Use bullet points and citation quotes to explain each insight.
If a fictional name cannot be inferred, default to the Reddit username: **{username}**

When presenting sliders (personality, motivations), use **visual bars**
made of characters (e.g. ▓▓▓░░░ 60%).

---

👤 **Name**:
real name based on comments. If none can be inferred, use: **{username}**

**Age Range**:
Estimate from context as close as possible (e.g. student, professional)

**Occupation**:
Inferred from tech interest, casual tone, or job-related phrases

**Status**:
(Optional – relationship/family indicators)

**Location**:
Infer from mentions of place or cultural context (e.g. Himachal, US, India)

**Tier**:
Early Adopter, Laggard, etc.

**Archetype**:
(e.g. The Skeptical Optimist, The Meme Monk, The Practical Tinkerer)

---

### 🧭 Personality Sliders
Use visual bars (like ▓ or █) and 0–100% estimates with short justifications. Example:

- **Introvert ↔ Extrovert**: ▓▓▓▓░░░░░ (40%)
  *"Posts infrequently and prefers comments to threads."*

- **Intuition ↔ Sensing**: ▓▓▓▓▓▓▓░░ (70%)
  *"Focuses on ideas, metaphors, and broader meaning."*

---

### 🎯 Motivations
Pick 4–6 motivations. For each:
- Name
- Intensity shown with visual bar (e.g. ▓▓▓▓▓▓░░ 75%)
- 1–2 Reddit quotes as evidence

---

### 🌀 Behaviours & Habits
4–6 bullet points, each with a matching quote. Examples:

- Shares screenshots of hikes or photos → *"Evening stroll. [4160x2340]"*
- Fixes SDK paths, updates Android configs → *"Update: changed ANDROID_HOME..."*

---

### 😤 Frustrations
3–5 frustrations. Include emotional tone + a quote for each.

---

### ✅ Goals & Needs
3–4 forward-facing goals inferred from struggles or desires.
No fluff — match with quotes if possible.

---

### 💬 Hero Quote
Choose a humorous, self-aware, or emotional quote from the user that sums them up.
Format:
> "quote here"

---

### 📚 Topics Discussed
Bullet points of recurring themes (e.g. Memes, Android SDK, Hiking, Java)

---

### 📌 Summary
Conclude with 2–3 lines that summarize who this person is, their vibe,
and what stands out.

---

Reddit content (posts and comments) from user u/{username}:

{joined_data}
"""

    return prompt


Generate Persona

In [44]:
# ✅ Define your API key at the top of your script, not inside the function
OPENROUTER_API_KEY = "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"


def generate_persona(prompt, retries=2):
    """
    Generate persona using OpenRouter API with retry logic.
    Uses the free MoonshotAI Kimi K2 model.
    """
    url = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://openrouter.ai",
        "X-Title": "RedditPersona"
    }

    payload = {
        "model": "moonshotai/kimi-k2:free",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 1000
    }

    for attempt in range(retries + 1):
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=60)

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]

            elif response.status_code == 408:
                print("⚠️ Timeout. Retrying...")
                time.sleep(5)

            elif response.status_code == 500:
                print("⚠️ Internal server error. Retrying...")
                time.sleep(5)

            elif response.status_code == 401:
                print("❌ Unauthorized — check API key.")
                print(response.text)
                return None

            else:
                print(f"❌ Error {response.status_code}")
                print(response.text)
                return None

        except requests.exceptions.Timeout:
            print("⚠️ Network timeout — retrying...")
            time.sleep(5)

    print("❌ Failed after all retries.")
    return None


Main Execution Pipeline

In [53]:
def main():
    """Execute the persona generation pipeline."""
    username = input("Enter Reddit username: ").strip()
    reddit_data = get_reddit_data(username)

    if reddit_data:
        prompt = build_prompt(username, reddit_data)
        persona = generate_persona(prompt)

        if persona:
            print("\n🎯 Persona Generated:\n")
            print(persona)

            # Optional: save to file
            filename = f"{username}_persona.txt"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(persona)
            print(f"\n✅ Saved to {filename}")
        else:
            print("❌ Persona generation failed.")
    else:
        print("❌ No Reddit data found.")


if __name__ == "__main__":
    main()


Enter Reddit username: _aadee_


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




🎯 Persona Generated:

👤 **Name**: **_aadee_**  
**Age Range**: 18-24 (college student vibe, “time pass kar lu thoda”)  
**Occupation**: Undergraduate / early-career, tech-curious hobbyist  
**Status**: Single, lives with parents  
**Location**: Himachal Pradesh, India (Shimla & nearby)  
**Tier**: Early Adopter (tries guitar, photography, new foods)  
**Archetype**: The Mountain Meme Monk – chilled, creative, joke-loving, rooted in the Himalayas.

---

### 🧭 Personality Sliders

- **Introvert ↔ Extrovert**: ▓▓▓░░░░░░ (30%)  
  *“Posts infrequently; comments are short jokes or emotes.”*

- **Intuition ↔ Sensing**: ▓▓▓▓▓▓░░░ (65%)  
  *“Focuses on mood, scenery, and symbolism (tattoo meaning, night lights).”*

- **Thinking ↔ Feeling**: ▓▓░░░░░░░ (25%)  
  *“Reacts with emojis and heart emotes rather than analysis.”*

- **Spontaneous ↔ Planner**: ▓▓▓▓▓░░░░ (55%)  
  *“Impulses: ‘bore ho raha tha toh aagya’.”*

---

### 🎯 Motivations

- **Creative Expression** ▓▓▓▓▓▓▓░░ (75%)  
  *“Shots 